In [ ]:
import os
import numpy as np
import requests
import rasterio
import cv2
from datetime import datetime
from pathlib import Path
from shapely.geometry import Polygon
from pystac_client import Client
from concurrent.futures import ThreadPoolExecutor
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuration - optimized for speed
TARGET_MONTH = "2020-06"
CITY = "Toronto"
PROVINCE = "Ontario"
COUNTRY = "Canada"
OUTPUT_DIR = "satellite_data"
CLOUD_COVERAGE_THRESHOLD = 30
MAX_WORKERS = 8
NDVI_THRESHOLD = 0.2
HIGHLIGHT_COLOR = [0, 255, 0]
HIGHLIGHT_ALPHA = 0.6

In [ ]:
class SatelliteProcessor:
    def __init__(self):
        self.base_dir = Path(OUTPUT_DIR)
        self.stac_client = Client.open("https://earth-search.aws.element84.com/v1")
        self.output_dir = self.base_dir / "outputs"
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.session = self._create_session()

    def _create_session(self):
        """Create HTTP session"""
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'SatelliteProcessor/1.0',
            'Connection': 'keep-alive'
        })
        return session

    def _s3_to_https(self, url):
        """S3 to HTTPS conversion"""
        if url.startswith('s3://'):
            return url.replace('s3://', 'https://').replace('/', '.s3.amazonaws.com/', 1)
        return url

    def _download_band(self, url):
        """Download band with debugging"""
        try:
            url = self._s3_to_https(url)
            print(f"       Downloading: {url[:80]}...")
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            print(f"       Downloaded {len(response.content)} bytes")
            
            with rasterio.MemoryFile(response.content) as memfile:
                with memfile.open() as src:
                    # Downsample for speed - use every 2nd pixel
                    data = src.read(1, out_shape=(src.height // 2, src.width // 2))
                    print(f"       Shape: {data.shape}, dtype: {data.dtype}")
                    return data.astype(np.float32)
        except Exception as e:
            print(f"       Error downloading: {e}")
            return None

    def _get_bounds(self, city, province, country):
        """Get city bounding box from Nominatim API"""
        try:
            address = f"{city}, {province}, {country}"
            response = requests.get(
                "https://nominatim.openstreetmap.org/search",
                params={'q': address, 'format': 'json', 'limit': 1},
                headers={'User-Agent': 'SatelliteProcessor/1.0'},
                timeout=15
            )
            
            data = response.json()
            if data and 'boundingbox' in data[0]:
                bbox = data[0]['boundingbox']
                return {
                    'min_lat': float(bbox[0]), 'max_lat': float(bbox[1]),
                    'min_lon': float(bbox[2]), 'max_lon': float(bbox[3])
                }
        except Exception as e:
            print(f"Error with location lookup: {e}")
        
        return None

    def _process_item(self, item):
        """Process single item with debugging"""
        try:
            print(f"   Processing: {item.id}")
            
            # Check available assets
            available_assets = list(item.assets.keys())
            print(f"     Available assets: {available_assets}")
            
            # Only download RGB + NIR - skip cloud detection
            bands = {}
            required = {'B04': 'red', 'B03': 'green', 'B02': 'blue', 'B08': 'nir'}
            
            # Try alternative asset names if standard ones don't exist
            asset_mapping = {
                'B04': ['B04', 'red', 'B4'],
                'B03': ['B03', 'green', 'B3'], 
                'B02': ['B02', 'blue', 'B2'],
                'B08': ['B08', 'nir', 'B8']
            }
            
            for standard_name, band_name in required.items():
                found = False
                for possible_name in asset_mapping[standard_name]:
                    if possible_name in item.assets:
                        print(f"     Found {possible_name} for {band_name}")
                        data = self._download_band(item.assets[possible_name].href)
                        if data is not None:
                            bands[band_name] = data
                            found = True
                            break
                
                if not found:
                    print(f"     Missing asset for {band_name}")
            
            print(f"     Downloaded bands: {list(bands.keys())}")
            
            # Need all 4 bands
            if len(bands) == 4:
                # Shape matching - resize to smallest
                shapes = [b.shape for b in bands.values()]
                min_h = min(s[0] for s in shapes)
                min_w = min(s[1] for s in shapes)
                print(f"     Target shape: {min_h}x{min_w}")
                
                for name in bands:
                    if bands[name].shape != (min_h, min_w):
                        bands[name] = cv2.resize(bands[name], (min_w, min_h))
                
                print(f"     ✅ Successfully processed {item.id}")
                return bands
            else:
                print(f"     ❌ Only got {len(bands)}/4 required bands")
                return None
                
        except Exception as e:
            print(f"     ❌ Error processing {item.id}: {e}")
            import traceback
            traceback.print_exc()
            return None

    def _create_composite(self, all_bands):
        """Create composite - simple median"""
        if not all_bands:
            return None
        
        composite = {}
        for band_name in ['red', 'green', 'blue', 'nir']:
            band_stack = [bands[band_name] for bands in all_bands if band_name in bands]
            if band_stack:
                # Simple median composite
                composite[band_name] = np.median(band_stack, axis=0).astype(np.float32)
        
        return composite if len(composite) == 4 else None

    def _create_output(self, bands):
        """Create visualization"""
        try:
            red, green, blue, nir = bands['red'], bands['green'], bands['blue'], bands['nir']
            
            # False color: NIR-Red-Green, simplified scaling
            false_color = np.stack([nir, red, green], axis=-1)
            
            # Normalization - use percentiles
            p1, p99 = np.percentile(false_color, [1, 99])
            false_color = np.clip((false_color - p1) / (p99 - p1), 0, 1)
            false_color = (false_color * 255).astype(np.uint8)
            
            # NDVI calculation
            ndvi = (nir - red) / (nir + red + 1e-8)
            vegetation_mask = ndvi > NDVI_THRESHOLD
            
            # Apply vegetation highlight
            if np.any(vegetation_mask):
                false_color[vegetation_mask] = cv2.addWeighted(
                    false_color[vegetation_mask], 
                    1 - HIGHLIGHT_ALPHA,
                    np.full_like(false_color[vegetation_mask], HIGHLIGHT_COLOR[::-1]),
                    HIGHLIGHT_ALPHA, 
                    0
                )
            
            veg_percent = np.sum(vegetation_mask) / vegetation_mask.size * 100
            return false_color, veg_percent
            
        except Exception as e:
            print(f"Error creating output: {e}")
            return None, 0

    def process(self):
        """Main processing pipeline"""
        start_time = time.time()
        
        print(f"🚀 Satellite Processor")
        print(f"📍 {CITY}, {PROVINCE}, {COUNTRY} | 📅 {TARGET_MONTH}")
        
        # Get bounds
        bounds = self._get_bounds(CITY, PROVINCE, COUNTRY)
        if not bounds:
            print("❌ Location not found")
            return
        
        # Date range
        year, month = map(int, TARGET_MONTH.split('-'))
        start_date = datetime(year, month, 1)
        end_date = datetime(year, month + 1, 1) if month < 12 else datetime(year + 1, 1, 1)
        
        # Query STAC - take first 10 items only
        print("🔍 Querying satellite data...")
        bbox = [bounds['min_lon'], bounds['min_lat'], bounds['max_lon'], bounds['max_lat']]
        search = self.stac_client.search(
            collections=["sentinel-2-l2a"],
            datetime=f"{start_date.isoformat()}/{end_date.isoformat()}",
            bbox=bbox,
            query={"eo:cloud_cover": {"lt": CLOUD_COVERAGE_THRESHOLD}},
            limit=10  # Limit for efficiency
        )
        
        items = list(search.items())
        print(f"📡 Found {len(items)} items")
        
        if not items:
            print("❌ No images found")
            return
        
        # Process items sequentially for better debugging
        print("⬇️ Processing items...")
        valid_results = []
        
        for i, item in enumerate(items[:5]):  # Process only first 5 for debugging
            print(f"\n--- Item {i+1}/5 ---")
            result = self._process_item(item)
            if result is not None:
                valid_results.append(result)
        
        print(f"\n✅ Processed {len(valid_results)}/{min(5, len(items))} items")
        
        if not valid_results:
            print("❌ No valid results")
            return
        
        # Create composite
        print("🌥️ Creating composite...")
        composite = self._create_composite(valid_results)
        
        if not composite:
            print("❌ Composite creation failed")
            return
        
        # Create final output
        print("🎨 Creating visualization...")
        output_image, veg_percent = self._create_output(composite)
        
        if output_image is None:
            print("❌ Visualization failed")
            return
        
        # Save result
        output_path = self.output_dir / f"{CITY}_{TARGET_MONTH}_vegetation.png"
        success = cv2.imwrite(str(output_path), output_image[..., ::-1])  # RGB to BGR
        
        elapsed = time.time() - start_time
        
        if success:
            print(f"✅ Saved: {output_path}")
            print(f"🌱 Vegetation: {veg_percent:.1f}%")
            print(f"⚡ Completed in {elapsed:.1f}s")
        else:
            print("❌ Save failed")

In [ ]:
def main():
    try:
        processor = SatelliteProcessor()
        processor.process()
    except Exception as e:
        print(f"❌ Error: {e}")

In [ ]:
if __name__ == "__main__":
    main()